In [84]:
import time
import requests
from bs4 import BeautifulSoup
import sys ,os,math
import json
import datetime

import io
import urllib
import http.cookiejar

word = '音乐'
is_chextend = 0  #### 文字拓展is_chextend = 1，语义拓展is_chextend = 0

In [85]:
t1 = time.time()

In [86]:
def competitive_product(word,appnum):
    url ='http://rest.appbk.com/app/get_all_app_search_results?n=' + word + '&start=0&limit=100&cc=cn' 
    appid = []
    try:
        resp_ = requests.get(url,timeout=timeout,verify = False)
        resp_2 = resp_.content
        resp_3 = resp_2.decode('utf-8')
        
        resp_list = json.loads(resp_3)
        resp_list = resp_list['results']
        for item in resp_list:
            appid.append(item['app_id']) 
            if len(appid) > appnum:
                break
        return appid    
    except Exception as e:
        print(e)

In [87]:
def competitive_product_kws_rank(appid_list,kwsnum):
    kws = []
    rank = []
    for appid in appid_list:       
        url ='http://rest.appbk.com/word/get_app_word_two_date_compare?email=&app_id=' + appid + '&token=&start=0&limit=12000&simple=1&cur_date=' + today + '&pre_date=' + yesterday        
        try:
            resp_ = requests.get(url,timeout=timeout,verify = False)
            resp_2 = resp_.content
            resp_3 = resp_2.decode('utf-8')

            resp_list = json.loads(resp_3)
            resp_list = resp_list['results']

            for item in resp_list:
                if item[0] not in kws and int(item[3])>4604:
                    kws.append(item[0]) 
                    rank.append(int(item[3])) 
                    if len(kws) > kwsnum:
                        break  
        except Exception as e:
            print(e)
    res = dict(zip(kws, rank))
    
    return sorted(res.items(), key=lambda d:d[1], reverse = True)
     


In [88]:
def relevance_qimai(word, inputword):
    url ='https://api.qimai.cn/trend/keywordExtend?analysis=LFRfBUNNMgMKIwQMcRpNF0tVC1BOCFJOR19FXXMZQVYPVXJAAVNRA1FRDwgCAVcjGgE%3D&keyword=' + word + '&page=1'
    try:
        data = {'username':'15652766069','password':'123456'}
        post_data = urllib.parse.urlencode(data).encode('utf-8')
        #设置请求头
        headers = {'User-agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'}
        #登录时表单提交到的地址（用开发者工具可以看到）
        login_url = 'https://api.qimai.cn/account/signinForm?analysis=IRIdAlMFDkMPFhZCWF4MCld2CkYMIxQGBQMODwVYBgVRCXJAAQ%3D%3D'
        session = requests.Session()
        resp = session.post(login_url,headers=headers, data=data)

        
        resp_ = session.get(url)
        resp_2 = resp_.content
        resp_3 = resp_2.decode('utf-8')        
        resp_list = json.loads(resp_3)
        resp_list = resp_list['extendList']['list']
        for item in resp_list:
            if item['word'] == inputword:
                return int(item['relate'])
    except Exception as e:
        print(e)
    return -1

In [89]:
today = datetime.date.today().strftime('%Y-%m-%d')
yesterday = (datetime.date.today() + datetime.timedelta(days=-1)).strftime('%Y-%m-%d')
now_time_H = datetime.datetime.now().strftime('%H')

timeout = 4

### 输入词下的前15个竞品appid
appid = competitive_product(word,10)

### 所有竞品关键词，每个竞品取热度前100的关键词
kws_rank = competitive_product_kws_rank(appid,100)

In [90]:
### 文字拓展筛选包含输入词的词
if is_chextend:
    kws_list = []
    for item in kws_rank:
        if word in item[0]:
            kws_list.append(item)
    kws_rank = kws_list

In [91]:
### 查询相关度
kws_list = []
for item in kws_rank:
    num = relevance_qimai(item[0], word)
    if num != -1:       
        temp = []
        temp.append(item[0])
        temp.append(item[1])
        temp.append(num/100)
        kws_list.append(temp)

In [92]:
kws_list

[['网易云音乐', 9504, 0.73],
 ['qq音乐', 9408, 0.72],
 ['酷狗音乐', 9400, 0.65],
 ['音乐', 8710, 1.0],
 ['虾米音乐', 7985, 0.7],
 ['酷我音乐', 7967, 0.71],
 ['酷我', 6795, 0.67],
 ['咪咕音乐', 6681, 0.69],
 ['apple', 6373, 0.09],
 ['Music', 6057, 0.48],
 ['apple muisc', 6006, 0.24],
 ['itunes音乐', 5750, 0.17],
 ['音乐app', 5133, 0.79],
 ['apple music', 5013, 0.44],
 ['酷我音乐盒', 5002, 0.65],
 ['天天动听', 4942, 0.23],
 ['多米音乐', 4900, 0.69],
 ['beats', 4799, 0.11],
 ['千千静听', 4704, 0.14],
 ['多米', 4664, 0.41],
 ['音樂', 4634, 0.8],
 ['天天动听音乐播放器', 4634, 0.37],
 ['applemusic', 4623, 0.17],
 ['咪咕彩铃', 4621, 0.68],
 ['kuwo', 4620, 0.66],
 ['itunes music', 4619, 0.1],
 ['joox music', 4617, 0.63],
 ['ipod', 4616, 0.16],
 ['咪咕爱唱', 4609, 0.65],
 ['appmusic', 4606, 0.52],
 ['酷狗音乐盒', 4606, 0.44],
 ['天天静听', 4606, 0.46],
 ['kugoo', 4606, 0.6],
 ['狗音乐', 4606, 0.68],
 ['咪咕音乐客户端', 4606, 0.69],
 ['米咕音乐', 4606, 0.68],
 ['音乐盒', 4606, 0.69],
 ['musica', 4605, 0.11],
 ['beats music', 4605, 0.16],
 ['畅想', 4605, 0.12],
 ['酷我音乐播放器', 4605, 0.47],
 ['音

In [93]:
time.time() - t1

47.14812612533569